In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
# Установка YOLOv5 и зависимостей
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17496, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 17496 (delta 2), reused 0 (delta 0), pack-reused 17491 (from 3)
Receiving objects: 100% (17496/17496), 16.56 MiB | 32.11 MiB/s, done.
Resolving deltas: 100% (11997/11997), done.
/kaggle/working/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.5 MB/s eta 0:00:0

In [ ]:
%pip install roboflow

In [ ]:
%pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="ZEDZtlyFJ62BJZEMiAyN")
project = rf.workspace("sigma-sht").project("petroglyphs-2oipt")
version = project.version(3)
dataset = version.download("yolov5-obb")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Petroglyphs-3 in yolov5-obb:: 100%|██████████| 419/419 [00:00<00:00, 4042.50it/s]


In [3]:
import os
os.listdir('/kaggle/working')


['petroglyph-annotation-1', '.virtual_documents']

In [6]:
weights_path = '/kaggle/input/moyamodelka/best.pt'

In [7]:
import os
print("Downloaded dataset path:", dataset.location)


Downloaded dataset path: /kaggle/working/yolov5/Petroglyphs-3


In [10]:
import yaml

data = {
    'path': '/kaggle/working/yolov5/Petroglyphs-3',
    'train': 'train/images',
    'val': 'valid/images',
    'nc': 1,
    'names': ['Petroglyph']
}

yaml_path = '/kaggle/working/yolov5/Petroglyphs-3/data.yaml'

with open(yaml_path, 'w') as f:
    yaml.dump(data, f, sort_keys=False)

print('✅ data.yaml создан в требуемом формате.')


✅ data.yaml создан в требуемом формате.


In [28]:
!python train.py \
  --weights /kaggle/input/moyamodelka/best.pt \
  --data /kaggle/working/Petroglyphs-3/data.yaml \
  --cache

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-07-02 04:31:54.163037: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751430714.186275     666 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751430714.193189     666 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&

In [27]:
import os
import cv2

input_folder = '/kaggle/working/Petroglyphs-3/valid/labelTxt'
output_folder = '/kaggle/working/Petroglyphs-3/valid/labels'
images_folder = '/kaggle/working/Petroglyphs-3/valid/images'

os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if not filename.endswith('.txt'):
        continue

    input_path = os.path.join(input_folder, filename)
    output_path = os.path.join(output_folder, filename)

    # Найти соответствующее изображение
    image_name = filename.replace('.txt', '.jpg')
    image_path = os.path.join(images_folder, image_name)

    if not os.path.exists(image_path):
        image_name = filename.replace('.txt', '.png')
        image_path = os.path.join(images_folder, image_name)

    if not os.path.exists(image_path):
        print(f"❌ Изображение для {filename} не найдено.")
        continue

    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ Ошибка чтения изображения {image_path}")
        continue

    IMAGE_HEIGHT, IMAGE_WIDTH = img.shape[:2]

    with open(input_path, 'r') as infile, open(output_path, 'w') as outfile:
        for line in infile:
            parts = line.strip().split()
            if len(parts) < 9:
                continue  # пропустить плохие строки

            x_coords = list(map(float, parts[:8:2]))
            y_coords = list(map(float, parts[1:8:2]))

            x_min = min(x_coords)
            x_max = max(x_coords)
            y_min = min(y_coords)
            y_max = max(y_coords)

            x_center = (x_min + x_max) / 2.0 / IMAGE_WIDTH
            y_center = (y_min + y_max) / 2.0 / IMAGE_HEIGHT
            width = (x_max - x_min) / IMAGE_WIDTH
            height = (y_max - y_min) / IMAGE_HEIGHT

            class_id = 0 

            outfile.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

print("✅ Конвертация завершена.")


✅ Конвертация завершена.
